Here's a list of problems that we still need to fix:
-Missing values. VISA has a decent amount and there's other missing values here and there. I think the best way would be to just take the previous value instead of replacing with the mean.

-Fixing return formula. Right now I just shift up 1 to get previous period values but this isn't valid if the row above is a different company.

-Have to reweight quarterly, not daily like how I have it


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
#Read in file of index changes
index_changes = pd.read_csv('index_changes.csv')
#Get only changes in the Dow
dow_changes = index_changes[index_changes['conm'] == 'Dow Jones Industrials-30 Stk']
#We need to rebalance annually. We will use October 1st as the rebalance date since most reconstitutions happen in late August
#or September. Thus we set the from date to be October 1st of the year that the stock joined the Dow
dow_changes['from'] = dow_changes['from'].apply(lambda x: dt.strptime(x, '%m/%d/%Y').replace(year=dt.strptime(x, '%m/%d/%Y').year, month=10, day=1).strftime('%m/%d/%Y'))
#If the end date is nan, change it to be 12/31/2021 (the last day in our date range)
dow_changes['thru'] = dow_changes['thru'].fillna('12/31/2021')
#Change all non 12/31/2021 'thru' dates to be October 1st (the reconstitution date)
dow_changes['thru'] = dow_changes['thru'].apply(lambda x: dt.strptime(x, '%m/%d/%Y').replace(year=dt.strptime(x, '%m/%d/%Y').year, month=10, day=1).strftime('%m/%d/%Y') if x != '12/31/2021' else x)


<ipython-input-2-a1d0a8697fa1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dow_changes['from'] = dow_changes['from'].apply(lambda x: dt.strptime(x, '%m/%d/%Y').replace(year=dt.strptime(x, '%m/%d/%Y').year, month=10, day=1).strftime('%m/%d/%Y'))
<ipython-input-2-a1d0a8697fa1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dow_changes['thru'] = dow_changes['thru'].fillna('12/31/2021')
<ipython-input-2-a1d0a8697fa1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [3]:
#Get the unique companies by gvkey
gvkeys = dow_changes['gvkey'].unique()
#Add the gvkeys to a .txt file to then be used as an input to get prices in Compustat
with open('gvkeys.txt', 'w') as f:
    for gvkey in gvkeys:
        f.write(str(gvkey) + '\n')

In [4]:
#Read in the returns data for only stocks that were part of the Dow over the specified time period
dow_returns = pd.read_csv('dow_stock_returns.csv')
#Turn the 'datadate' column into datetime type
dow_returns['datadate'] = pd.to_datetime(dow_returns['datadate'], format="%m/%d/%Y")
#Determine which quarter each date falls under
dow_returns['quarter'] = dow_returns['datadate'].dt.quarter
#Turn the 'from' and 'thru' columns into datetime types
dow_changes['from'] = pd.to_datetime(dow_changes['from'], format="%m/%d/%Y")
dow_changes['thru'] = pd.to_datetime(dow_changes['thru'], format="%m/%d/%Y")
#Get only return data for stocks that were in the Dow and not before they joined or after they dropped out
dow_returns = dow_returns.merge(dow_changes, on=['gvkey','iid'], how='left')
dow_returns = dow_returns[dow_returns['datadate']>=dow_returns['from']]
dow_returns = dow_returns[dow_returns['datadate']<=dow_returns['thru']]
#Make sure the dates are only on or after 12/31/2021 and on or before 12/31/2021
dow_returns = dow_returns[dow_returns['datadate']>=dt.strptime('12/31/2011', '%m/%d/%Y')]
dow_returns = dow_returns[dow_returns['datadate']<=dt.strptime('12/31/2021', '%m/%d/%Y')]
    

<ipython-input-4-ebe0c30e7eca>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dow_changes['from'] = pd.to_datetime(dow_changes['from'], format="%m/%d/%Y")
<ipython-input-4-ebe0c30e7eca>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dow_changes['thru'] = pd.to_datetime(dow_changes['thru'], format="%m/%d/%Y")


In [15]:
#Get the adjusted close price
dow_returns['adj_close'] = dow_returns['prccd']/dow_returns['ajexdi']
#Check that the previous row corresponds to the same stock so that we don't accidentally use another stock's date to calculate returns
if dow_returns['gvkey'] == dow_returns['gvkey'].shift(1):
    print('test')
    #Compute the stock return
    #dow_returns['stock_return'] = ((((dow_returns['adj_close'])*dow_returns['trfd'])/(dow_returns['adj_close'].shift(1)*dow_returns['trfd'].shift(1)))-1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
#Get the weight of each stock for each day
dow_returns['stock_weight'] = dow_returns.groupby('datadate')['adj_close'].transform(lambda x: x/x.sum())
#Find the contribution to the Dow's aggregate return
dow_returns['weighted_return'] = dow_returns['stock_return']*dow_returns['stock_weight']
dow_agg = dow_returns.groupby('datadate')['adj_close','weighted_return'].sum()

<ipython-input-6-eb32b05a3457>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dow_agg = dow_returns.groupby('datadate')['adj_close','weighted_return'].sum()


In [ ]:
dow_returns[dow_returns['datadate']=='12/30/2021']

,gvkey,iid,datadate,conm_x,ajexdi,prccd,trfd,quarter,gvkeyx,from,thru,conm_y,tic,adj_close,stock_return,stock_weight,weighted_return
3019,1300,1,2021-12-30,HONEYWELL INTERNATIONAL INC,1.0,207.11,2.965164,4,5.0,2020-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,207.11,-0.002024,0.037502,-0.000076
6237,1447,1,2021-12-30,AMERICAN EXPRESS CO,1.0,164.16,2.653703,4,5.0,1982-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,164.16,0.002014,0.029725,0.000060
9445,1602,1,2021-12-30,AMGEN INC,1.0,226.47,1.292331,4,5.0,2020-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,226.47,-0.004965,0.041008,-0.000204
12653,1690,1,2021-12-30,APPLE INC,1.0,178.20,1.278096,4,5.0,2015-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,178.20,-0.006578,0.032267,-0.000212
15861,2136,1,2021-12-30,VERIZON COMMUNICATIONS INC,1.0,52.25,6.746876,4,5.0,2004-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,52.25,-0.002101,0.009461,-0.000020
19069,2285,1,2021-12-30,BOEING CO,1.0,202.71,2.175472,4,5.0,1987-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,202.71,-0.004665,0.036706,-0.000171
22277,2817,1,2021-12-30,CATERPILLAR INC,1.0,206.08,2.380727,4,5.0,1991-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,206.08,-0.006029,0.037316,-0.000225
25485,2968,1,2021-12-30,JPMORGAN CHASE & CO,1.0,158.48,4.264388,4,5.0,2001-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,158.48,-0.000505,0.028697,-0.000014
45928,2991,1,2021-12-30,CHEVRON CORP,1.0,117.43,4.635735,4,5.0,2008-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,117.43,-0.004409,0.021264,-0.000094
49136,3144,1,2021-12-30,COCA-COLA CO,1.0,58.78,2.585079,4,5.0,1932-10-01,2021-12-31,Dow Jones Industrials-30 Stk,I0005,58.78,-0.002884,0.010644,-0.000031
